# Power Outage Analysis

**Name(s)**: Walter White, Nathaniel Bacon

**Website Link**: (your website link)

## Code

In [ ]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

In [ ]:
outage_df = pd.read_excel('outage.xlsx')
# outage_df.loc[4:]
test_df = pd.DataFrame(columns=outage_df.iloc[4].tolist()[1:])
test_df

# Introduction and Question Identification

This dataset has major power outage data in the continental U.S. from January 2000 to July 2016, provided by Laboratory for Advancing Sustainable Critical Infrastructure (LASCI)

Question: does month have an effect on the severity of power outages? 

Null hypothesis: The summer months (7, 8, 9) have the same average number of customers affected as the rest of the year.

Alternate hypothesis: the summer months have greater power outages than rest of the year

We use mean to do permutation test
Accounting for outliers of both sets, but outliers are often urban centers which are important




1534 rows, 54 columns

Relevant columns:
- `U.S._STATE`: Represents all the states in the continental U.S.
- `YEAR`: Indicates the year when the outage event occurred
- `MONTH`: Indicates the month when the outage event occurred
- `CUSTOMERS.AFFECTED`: Number of customers affected by the power outage event
- `CLIMATE.REGION`: U.S. Climate regions as specified by National Centers for Environmental Information (nine climatically consistent regions in continental U.S.A.)
- `DEMAND.LOSS.MW`: Amount of peak demand lost during an outage event (in Megawatt) [but in many cases, total demand is reported]
- `TOTAL.PRICE`: Average monthly electricity price in the U.S. state (cents/kilowatt-hour)
- `TOTAL.CUSTOMERS`: Annual number of total customers served in the U.S. state
- `POPULATION`: Population in the U.S. state in a year

In [ ]:
#CLEANING
a = dict(zip(list(pd.DataFrame(outage_df.iloc[5:, 1:]).columns), outage_df.iloc[4].tolist()[1:]))
outage_cleaned = pd.DataFrame(outage_df.iloc[6:, 1:]).rename(columns=a).reset_index().drop(['index'], axis='columns')
outage_cleaned['OUTAGE.START'] = (outage_cleaned['OUTAGE.START.DATE'].transform(lambda x: str(x).split(' ')[0]) + ' ' +  outage_cleaned['OUTAGE.START.TIME'].apply(str)).apply(lambda x: np.NAN if 'nan' in x else pd.to_datetime(x).to_pydatetime())
outage_cleaned['OUTAGE.RESTORATION'] = (outage_cleaned['OUTAGE.RESTORATION.DATE'].transform(lambda x: str(x).split(' ')[0]) + ' ' +  outage_cleaned['OUTAGE.RESTORATION.TIME'].apply(str)).apply(lambda x: np.NAN if 'nan' in x else pd.to_datetime(x).to_pydatetime())
outage_cleaned = outage_cleaned.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], axis='columns')
outage_cleaned.head()

1. Formatted the excel file to adjust into proper DataFrame format for ease of access and readability
    - Take the 
2. Parsed data for Date and Time into 'OUTAGE.START' and 'OUTAGE.RESTORATION':
    - Dropped redundant columns 




# Univariate Analysis


In [40]:
df = outage_cleaned.copy()
df = df.rename(columns={'CLIMATE.REGION': 'Power Outage'})

df = df[['Power Outage']]
fig = px.histogram(df)
fig.update_layout(
    title_text="Power Outages by Region",
    xaxis_title="Climate Regions",
    yaxis_title="Number",
)
fig.show()

### Explanation of Graph: 'Power Outages by Region'
This is a graph displaying the distributions of power outages within climate regions in the U.S. 

It is evident that the Northeast region has the highest number of power outages, totaling 350, while the West North Central region has the least, with fewer than 20 outages. For the most part, the other regions fall between 100 to 250 outages.




In [47]:
month_count = outage_cleaned.groupby('MONTH').count()

fig = px.bar(month_count, y = 'OBS')
fig.update_layout(
    title_text="Power Outages by Month",
    xaxis_title="Month",
    yaxis_title="Outage Count"
)
fig.show()

### Explanation of Graph: 'Power Outages by Month'
This is a graph displaying the distributions of power outages within each month.

Power outages are most prevalent during the summer, with a noticeable decease compared to the winter months. While there is a significant number of outages in winter, it doesn't compare to the frequency experienced during the summer season.







# Bivariate Analysis


In [75]:
df = outage_cleaned.copy()
state_count = outage_cleaned.groupby('U.S._STATE')['POPULATION'].transform('mean')
df['pop_mean'] = state_count
state_means = df.groupby('U.S._STATE')['TOTAL.CUSTOMERS'].mean()/df.groupby('U.S._STATE').mean()['pop_mean']  
fig = px.bar(state_means)
fig.update_layout(
    title_text="State Proportion of Customers",
    xaxis_title="State",
    yaxis_title="Proportion of customers"
)
fig.show()

### Explanation of Graph: 'State Proportions of Customers'
This graph displays the proportion of customers to the population mean over the years of each state.

There isn't particularly any state that is a significant outlier. Hawaii and Maine have a small customer base while Maine has a surprising amount. 


In [ ]:
#MAR test
#take states
#make columns missing=True and missing=False for customers_affected missing or not
#groupby, take counts of true/false columns fuck pivot table
mar_test = outage_cleaned.copy()
mar_test['missingTrue'] = mar_test['CUSTOMERS.AFFECTED'].isna()
mar_test['missingFalse'] = mar_test['CUSTOMERS.AFFECTED'].notna()
observed_tvd = (mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2

tvds = []
for _ in range(100):
    mar_test['U.S._STATE'] = np.random.permutation(mar_test['U.S._STATE'])
    perm_tvd = (mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2
    tvds.append(perm_tvd)
np.mean(tvds > observed_tvd)
# tvds

### Interesting Aggregates	

In [76]:
grouped_df = outage_cleaned.groupby('U.S._STATE').agg({'TOTAL.PRICE': 'mean', 'TOTAL.CUSTOMERS': 'mean'}).reset_index()


df = grouped_df.copy()


fig = px.scatter(df, x='TOTAL.PRICE', y='TOTAL.CUSTOMERS', text='U.S._STATE', title='Mean Price vs Mean Customers per State')


fig.show()

'Mean Price vs Mean Customers per State' reveals the outliers of data set. Hawaii is outlandishly expensive despite have a small number of customers. California has a massively large customer base. 

# Assessment of Missingness


#### NMAR Analysis
Column that we think are NMAR:
 - Cause Category Detail
'CAUSE.CATEGORY.DETAIL' is a detailed description of the event categories causing the major power outages. It is NMAR is because the cause doesnt fit in one of the limited categories either being an unknown cause or being a multitude of causes. Having a column 'DETAIL_MISSING', indicating whether or not the 'CAUSE.CATEGORY.DETAIL' is missing or not, would shift it from NMAR to MAR. 


 

#### Missingness Dependency	


 MAR: state vs customers missing



In [92]:
#MAR test
#take states
#make columns missing=True and missing=False for customers_affected missing or not
#groupby, take counts of true/false columns fuck pivot table
mar_test = outage_cleaned.copy()
mar_test['missingTrue'] = mar_test['CUSTOMERS.AFFECTED'].isna()
mar_test['missingFalse'] = mar_test['CUSTOMERS.AFFECTED'].notna()
observed_tvd = (mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2

tvds = []
for _ in range(100):
    mar_test['U.S._STATE'] = np.random.permutation(mar_test['U.S._STATE'])
    perm_tvd = (mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2
    tvds.append(perm_tvd)
np.mean(tvds > observed_tvd)
observed_tvd


/var/folders/nt/trr1v_vd75qb1zyf0b0xzkzc0000gn/T/ipykernel_60094/1559120592.py:8: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/var/folders/nt/trr1v_vd75qb1zyf0b0xzkzc0000gn/T/ipykernel_60094/1559120592.py:13: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



0.3791869864870177

In [91]:
fig = px.histogram(tvds)
if fig.data:
    fig.update_layout(
        shapes=[
            dict(
                type='line',
                x0=observed_tvd,
                x1=observed_tvd,
                y0=0,
                y1 = 1,
                line=dict(color='red', width=2)
            )
        ]
    )

fig.show()

Number of missing values per column

In [ ]:
a = dict(zip(list(pd.DataFrame(outage_df.iloc[5:, 1:]).columns), outage_df.iloc[4].tolist()[1:]))
outage_cleaned = pd.DataFrame(outage_df.iloc[6:, 1:]).rename(columns=a).reset_index().drop(['index'], axis='columns')
outage_cleaned[outage_cleaned.isna().any(axis=1)]
outage_cleaned.T.isna().sum(axis=1).sort_values(ascending=False)


In [ ]:
outage_cleaned[outage_cleaned['IND.PRICE'].isna()]

Combined start date and time into one column 'start'

In [ ]:
# to delete
#  outage_cleaned['OUTAGE.START.TIME'][0] + outage_cleaned['OUTAGE.START.DATE'][0]
# outage_cleaned['OUTAGE.START.DATE'][0].date()
# outage_cleaned['OUTAGE.START.DATE'].transform(lambda x: x.replace(hour=1, minute=10))
# outage_cleaned['OUTAGE.START.TIME'][0].hour

test_copy = outage_cleaned.copy()
# test_copy['OUTAGE.START.DATE'].dt.date + pd.to_timedelta(test_copy['OUTAGE.START.TIME'].dt.strftime('%H:%M:%S'))
test_copy['OUTAGE.START.TIME'][0].hour
# new_date = pd.datetime(test_copy['OUTAGE.START.DATE'][0].year, test_copy['OUTAGE.START.DATE'][0].month, test_copy['OUTAGE.START.DATE'][0].day, test_copy['OUTAGE.START.TIME'][0].hour, test_copy['OUTAGE.START.TIME'][0].minute, 0)

# test_copy['test_date'] = test_copy.apply(lambda x, y: pd.datetime(x.year, x.month, x.date, y.hour, y.minute, 0), test_copy['OUTAGE.START.DATE'], test_copy['OUTAGE.START.TIME'])
# pd.to_datetime(test_copy['OUTAGE.START.TIME'][0])
# datetime_object = datetime.combine(datetime.date.today(), time)

pd.datetime.combine(pd.datetime.today().date(), test_copy['OUTAGE.START.TIME'][0])
test_copy['OUTAGE.START.TIME'] = test_copy['OUTAGE.START.TIME'].apply(lambda x: pd.datetime.combine(pd.datetime.today().date(), x) if type(x) != float else pd.datetime(2023, 1, 1, 0, 0, 0))
test_copy['OUTAGE.START.TIME'][0]
test_copy['OUTAGE.START.DATE'].dt.date + pd.to_timedelta(test_copy['OUTAGE.START.TIME'])#.dt.strftime('%H:%M:%S'))


Combined restoration date and time into one column 'restoration start'

In [ ]:
# to delete
test_copy = outage_cleaned.copy()
# test_copy['OUTAGE.START.DATE'][0].hour = 1
# test_copy['test_date'] = test_copy.apply(lambda x: pd.datetime(x.loc['OUTAGE.START.DATE'].year, x.loc['OUTAGE.START.DATE'].month, x.loc['OUTAGE.START.DATE'].date, x.loc['OUTAGE.START.TIME'].hour, x.loc['OUTAGE.START.TIME'].minute, 0))
# pd.datetime.combine(test_copy['OUTAGE.START.DATE'][0].date(), test_copy['OUTAGE.START.TIME'][0])
# test_copy['test_date'] = test_copy.apply(lambda x: True if x['U.S._STATE']=='Minnesota' else False)
# test_copy
# test_copy['OUTAGE.START.DATE'][0] = pd.to_datetime(str(test_copy['OUTAGE.START.DATE'][0]).split(' ')[0] + ' ' + str(test_copy['OUTAGE.START.TIME'][0]))

test_copy['OUTAGE.START'] = (test_copy['OUTAGE.START.DATE'].transform(lambda x: str(x).split(' ')[0]) + ' ' +  test_copy['OUTAGE.START.TIME'].apply(str)).apply(lambda x: np.NAN if 'nan' in x else pd.to_datetime(x).to_pydatetime())
test_copy['OUTAGE.RESTORATION'] = (test_copy['OUTAGE.RESTORATION.DATE'].transform(lambda x: str(x).split(' ')[0]) + ' ' +  test_copy['OUTAGE.RESTORATION.TIME'].apply(str)).apply(lambda x: np.NAN if 'nan' in x else pd.to_datetime(x).to_pydatetime())
test_copy = test_copy.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], axis='columns')

In [ ]:
# type(test_copy.iloc[-1]['OUTAGE.START.DATE'])
test_copy.tail(5)

In [ ]:
outage_cleaned

In [ ]:
pd.set_option('max_column', None)
# outage_cleaned[outage_cleaned['IND.PRICE'].isna()]
# outage_cleaned[outage_cleaned['OUTAGE.DURATION'].isna()]

 MAR: state vs customers missing

 NMAR: demand loss, cause category detail
 
 just do cause category, missing because cannot properly fit into 

In [ ]:
#MAR test
#take states
#make columns missing=True and missing=False for customers_affected missing or not
#groupby, take counts of true/false columns fuck pivot table
mar_test = outage_cleaned.copy()
mar_test['missingTrue'] = mar_test['CUSTOMERS.AFFECTED'].isna()
mar_test['missingFalse'] = mar_test['CUSTOMERS.AFFECTED'].notna()
observed_tvd = (mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2

tvds = []
for _ in range(100):
    mar_test['U.S._STATE'] = np.random.permutation(mar_test['U.S._STATE'])
    perm_tvd = (mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2
    tvds.append(perm_tvd)
np.mean(tvds > observed_tvd)
# tvds

Find observed statistic, which is mean of customers affected of 7,8,9 minus mean of all months

Then shuffle 'month' column 1000 times, find the difference, find percentage of those that had higher difference than observed

In [ ]:
#question: does month have an effect on the severity of power outages?
#null hypothesis: The summer months (7, 8, 9) have the same average number of customers affected as the rest of the year.
#alternate hypothesis: the summer months have greater power outages than rest of the year
#note: use mean to do permutation test
#accounting for outliers of both sets, but outliers are often urban centers which are important

#find observed statistic, which is mean of customers affected of 7,8,9 minus mean of all months
# shuffle month column 1000 times, find diff, find percentage of those that had higher diff than observed
outage_cleaned[(outage_cleaned['MONTH'] == 7) | (outage_cleaned['MONTH'] == 8) | (outage_cleaned['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].median()

In [ ]:
observed_diff = outage_cleaned[(outage_cleaned['MONTH'] == 7) | (outage_cleaned['MONTH'] == 8) | (outage_cleaned['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].median() - outage_cleaned['CUSTOMERS.AFFECTED'].median()

outage_copy = outage_cleaned.copy()
p_vals = []
diffs_list = []
for __ in range(100):
    for _ in range(1000):
        outage_copy['MONTH'] = np.random.permutation(outage_copy['MONTH'])
        perm_diff = outage_copy[(outage_copy['MONTH'] == 7) | (outage_copy['MONTH'] == 8) | (outage_copy['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].median() - outage_copy['CUSTOMERS.AFFECTED'].median()
        diffs_list.append(perm_diff)
    p_vals.append(np.mean(diffs_list > observed_diff))
np.mean(p_vals)

In [ ]:
#question: does month have an effect on the severity of power outages?
#null hypothesis: The summer months (7, 8, 9) have the same average number of customers affected as the rest of the year.
#alternate hypothesis: the summer months have greater power outages than rest of the year
#note: use mean to do permutation test
#accounting for outliers of both sets, but outliers are often urban centers which are important

#find observed statistic, which is mean of customers affected of 7,8,9 minus mean of all months
# shuffle month column 1000 times, find diff, find percentage of those that had higher diff than observed

observed_diff = outage_cleaned[(outage_cleaned['MONTH'] == 7) | (outage_cleaned['MONTH'] == 8) | (outage_cleaned['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].mean() - outage_cleaned['CUSTOMERS.AFFECTED'].mean()

outage_copy = outage_cleaned.copy()
p_vals = []
diffs_list = []
for __ in range(100):
    for _ in range(1000):
        outage_copy['MONTH'] = np.random.permutation(outage_copy['MONTH'])
        perm_diff = outage_copy[(outage_copy['MONTH'] == 7) | (outage_copy['MONTH'] == 8) | (outage_copy['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].mean() - outage_copy['CUSTOMERS.AFFECTED'].mean()
        diffs_list.append(perm_diff)
    p_vals.append(np.mean(diffs_list > observed_diff))
np.mean(p_vals)

### Cleaning and EDA

### Assessment of Missingness

In [ ]:
# TODO

### Hypothesis Testing

In [ ]:
# the time of the year has no effect on the severity of power outages (how many customers affected)